In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules

dat = pd.read_csv('website_link.csv')
dat = dat.drop(columns = ['Unnamed: 0'])
print(len(dat))

48674


In [2]:
# Getting the unique id for each of the url
alist = []
unique_id = []
for i in range(len(dat['links'])):
    url = dat['links'][i].replace(" ", "")
    if str(url) not in alist:
        alist.append(str(url))
        unique_id.append(len(alist)-1)
    else:
        val = alist.index(str(url))
        unique_id.append(val)
        
#print(len(alist))    #28295
#print(len(unique_id))   #48674
df_id = pd.DataFrame({'id': unique_id})
df = pd.concat([dat, df_id], axis= 1)
df.tail(10)

,links,product,id
48664,http://www.amazon.co.uk/Star-Wars-Costume-Sto...,"Star Wars Costume, Kids Stormtrooper Costume S...",28287
48665,http://www.amazon.co.uk/Rubies-Costume-Star-W...,"Star Wars Costume, Kids Stormtrooper Costume S...",28288
48666,http://www.amazon.co.uk/STAR-WARS-Stormtrooper,"Star Wars Costume, Kids Stormtrooper Costume S...",28289
48667,http://www.amazon.co.uk/Deluxe-Stormtrooper,"Star Wars Costume, Kids Stormtrooper Costume S...",10801
48668,http://www.amazon.co.uk/Bandai-Star-Wars-Fight...,Star Wars 1/72 Y-Wing Starfighter,28290
48669,http://www.amazon.co.uk/Bandai-Star-Wars-Star...,Star Wars 1/72 Y-Wing Starfighter,28291
48670,http://www.amazon.co.uk/Star-Wars-48-Snow-Spe...,Star Wars 1/72 Y-Wing Starfighter,28292
48671,http://www.amazon.co.uk/Bandai-Advanced-Starf...,Star Wars 1/72 Y-Wing Starfighter,28293
48672,http://www.amazon.co.uk/Star-Wars-fighter-res...,Star Wars 1/72 Y-Wing Starfighter,28294
48673,http://www.amazon.co.uk/Millennium,Star Wars 1/72 Y-Wing Starfighter,3455


In [3]:
"""
df['id'].nunique()
df['id'].value_counts().sort_values(ascending=False).head(10)
"""

"\ndf['id'].nunique()\ndf['id'].value_counts().sort_values(ascending=False).head(10)\n"

In [4]:
"""
import matplotlib.pyplot as plt
fig, ax=plt.subplots(figsize=(6,4))
df['id'].value_counts().sort_values(ascending=False).head(10).plot(kind='bar')
plt.ylabel('Number of transactions')
plt.xlabel('Items')
ax.get_yaxis().get_major_formatter().set_scientific(False)
plt.title('Best sellers')
"""

"\nimport matplotlib.pyplot as plt\nfig, ax=plt.subplots(figsize=(6,4))\ndf['id'].value_counts().sort_values(ascending=False).head(10).plot(kind='bar')\nplt.ylabel('Number of transactions')\nplt.xlabel('Items')\nax.get_yaxis().get_major_formatter().set_scientific(False)\nplt.title('Best sellers')\n"

In [5]:
"""
lst = []
for item in df['product'].unique():
    lst2=list(set(df[df['product']==item]['id']))
    if len(lst2)>0:
        lst.append(lst2)
print(lst[0:10])
"""

"\nlst = []\nfor item in df['product'].unique():\n    lst2=list(set(df[df['product']==item]['id']))\n    if len(lst2)>0:\n        lst.append(lst2)\nprint(lst[0:10])\n"

In [6]:
"""
te=TransactionEncoder()
te_data=te.fit(lst).transform(lst)
data_x=pd.DataFrame(te_data,columns=te.columns_)

frequent_items= apriori(data_x, use_colnames=True, min_support=0.002)
print(frequent_items)

rules = association_rules(frequent_items, metric="confidence", min_threshold=0.7)
rules.antecedents = rules.antecedents.apply(lambda x: next(iter(x)))
rules.consequents = rules.consequents.apply(lambda x: next(iter(x)))
rules
"""

'\nte=TransactionEncoder()\nte_data=te.fit(lst).transform(lst)\ndata_x=pd.DataFrame(te_data,columns=te.columns_)\n\nfrequent_items= apriori(data_x, use_colnames=True, min_support=0.002)\nprint(frequent_items)\n\nrules = association_rules(frequent_items, metric="confidence", min_threshold=0.7)\nrules.antecedents = rules.antecedents.apply(lambda x: next(iter(x)))\nrules.consequents = rules.consequents.apply(lambda x: next(iter(x)))\nrules\n'

In [7]:
df['id'] = df['id'].apply(str)

In [8]:
# Same product concat it into one record, required format for apriori
new_df = df[['product', 'id']].groupby(['product']).transform(lambda x: ','.join(x))
new_df['product'] = df['product']

In [9]:
new_df.drop_duplicates(subset='id', inplace=True)
new_df.reset_index(drop=True, inplace=True)
print(len(new_df))

8711


In [10]:
new_df.head(50)
new_df.rename(columns={'id': "linked_prod_list"}, inplace=True)
new_df.head(10)

,linked_prod_list,product
0,"0,1,2,3,4,5",Hornby 2014 Catalogue
1,"6,7,8,9,10,11",FunkyBuys® Large Christmas Holiday Express Fes...
2,"12,13,14,15,16,17",CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...
3,"18,19,20,21,22,23",Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...
4,"24,25,26,27,28",20pcs Model Garden Light Double Heads Lamppost...
5,"29,30",Hornby 00 Gauge 230mm BR Bogie Passenger Brake...
6,"31,20,23,32,33,34",Hornby Santa's Express Train Set
7,"35,4,36,37,38,39",Hornby Gauge Western Express Digital Train Set...
8,"40,41,40,42",Learning Curve Chuggington Interactive Chatsworth
9,"43,20,44,45,46,47",Hornby Gauge Railroad Mosley Tarmacadam Locomo...


In [11]:
# Rearrange the format
for t in range(len(new_df)):
    result = new_df['linked_prod_list'][t].split(",")
    new_df['linked_prod_list'][t] = result

new_df['linked_prod_list']

0                               [0, 1, 2, 3, 4, 5]
1                             [6, 7, 8, 9, 10, 11]
2                         [12, 13, 14, 15, 16, 17]
3                         [18, 19, 20, 21, 22, 23]
4                             [24, 25, 26, 27, 28]
5                                         [29, 30]
6                         [31, 20, 23, 32, 33, 34]
7                          [35, 4, 36, 37, 38, 39]
8                                 [40, 41, 40, 42]
9                         [43, 20, 44, 45, 46, 47]
10                        [48, 49, 50, 51, 52, 53]
11                        [54, 55, 56, 57, 58, 59]
12                        [60, 61, 62, 63, 64, 65]
13                        [41, 41, 41, 66, 41, 41]
14                                [67, 68, 69, 70]
15                        [71, 72, 73, 74, 75, 76]
16                                        [77, 78]
17                                        [79, 80]
18                        [81, 82, 83, 84, 85, 30]
19                        [86, 

In [12]:
new_df.head(10)

,linked_prod_list,product
0,"[0, 1, 2, 3, 4, 5]",Hornby 2014 Catalogue
1,"[6, 7, 8, 9, 10, 11]",FunkyBuys® Large Christmas Holiday Express Fes...
2,"[12, 13, 14, 15, 16, 17]",CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...
3,"[18, 19, 20, 21, 22, 23]",Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...
4,"[24, 25, 26, 27, 28]",20pcs Model Garden Light Double Heads Lamppost...
5,"[29, 30]",Hornby 00 Gauge 230mm BR Bogie Passenger Brake...
6,"[31, 20, 23, 32, 33, 34]",Hornby Santa's Express Train Set
7,"[35, 4, 36, 37, 38, 39]",Hornby Gauge Western Express Digital Train Set...
8,"[40, 41, 40, 42]",Learning Curve Chuggington Interactive Chatsworth
9,"[43, 20, 44, 45, 46, 47]",Hornby Gauge Railroad Mosley Tarmacadam Locomo...


In [13]:
# Evaluate the frequency of each item (affects support)
import collections
import operator
import matplotlib.pyplot as plt
content_list = [item for sublist in new_df['linked_prod_list'] for item in sublist]
counts_all = collections.Counter(content_list)
words, count_values = zip(*counts_all.items())
values_sorted, words_sorted = zip(*sorted(zip(count_values, words), key=operator.itemgetter(0), reverse=True))
words_sorted_top = words_sorted[0:10]
values_sorted_top = values_sorted[0:10]
plt.bar(words_sorted_top,values_sorted_top,align='center') # A bar chart
plt.show()

<Figure size 640x480 with 1 Axes>

In [14]:
# Pass the url represented by unique IDs into the apriori algorithm
# Encode the data into a sparse dataframe
te = TransactionEncoder()
te_array = te.fit(new_df['linked_prod_list']).transform(new_df['linked_prod_list'], sparse=True)
convert = pd.SparseDataFrame(te_array, columns=te.columns_, default_fill_value=False)
convert

# Apriori algorithm to obtain the frequent itemset
fr_itemsets = apriori(convert, min_support=0.002, use_colnames= True)
fr_itemsets['length'] = fr_itemsets['itemsets'].apply(lambda x: len(x))
fr_itemsets

,support,itemsets,length
0,0.004362,(1038),1
1,0.002181,(1040),1
2,0.015957,(1238),1
3,0.017220,(1265),1
4,0.002411,(12811),1
5,0.005625,(1300),1
6,0.002870,(1301),1
7,0.003559,(1303),1
8,0.004133,(1304),1
9,0.002181,(1353),1


In [15]:
# Obtain the rules that fulfills the min confidence
rules = association_rules(fr_itemsets, metric="confidence", min_threshold = 0.7)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(1301),(1300),0.002870,0.005625,0.002411,0.840000,149.331429,0.002395,6.214843
1,(1303),(1300),0.003559,0.005625,0.003214,0.903226,160.571429,0.003194,10.275208
2,(1304),(1300),0.004133,0.005625,0.003788,0.916667,162.960884,0.003765,11.932499
3,(1301),(1304),0.002870,0.004133,0.002066,0.720000,174.220000,0.002054,3.556669
4,(1303),(1304),0.003559,0.004133,0.002870,0.806452,195.138889,0.002855,5.145314
5,"(1304, 1301)",(1300),0.002066,0.005625,0.002066,1.000000,177.775510,0.002055,inf
6,"(1300, 1301)",(1304),0.002411,0.004133,0.002066,0.857143,207.404762,0.002056,6.971071
7,(1301),"(1300, 1304)",0.002870,0.003788,0.002066,0.720000,190.058182,0.002055,3.557899
8,"(1303, 1304)",(1300),0.002870,0.005625,0.002755,0.960000,170.664490,0.002739,24.859373
9,"(1300, 1304)",(1303),0.003788,0.003559,0.002755,0.727273,204.363636,0.002742,3.653618


In [22]:
new_list = []
for i in rules['antecedents'].unique():
    x = i
    string = ""
    for t in x:
        print(t)
        if len(string) < 4:
            string += t
    if string not in new_list:
        new_list.append(string)

print(new_list)

1301
1303
1304
1304
1301
1300
1301
1303
1304
1300
1304
1303
1300
['1301', '1303', '1304', '1300']


In [23]:
url_list = []
for i in new_list:
    val = int(i)
    url_list.append(alist[val])
print(url_list)

['http://www.amazon.co.uk/Dungeon-Masters-Dungeons-Dragons-Accessories', 'http://www.amazon.co.uk/Monster-Manual-Dungeons-Rulebook-Rulebooks/dp/0786965614', 'http://www.amazon.co.uk/Dungeon-Masters-Dungeons-Dragons-Rulebooks/dp/0786965622', 'http://www.amazon.co.uk/Dungeons-Dragons-Players-Handbook-Rulebooks/dp/0786965606']


In [24]:
# Requests for the product name through web scraping
import requests
import urllib.request
from bs4 import BeautifulSoup
import time

def get_product(url):
    print(url)
    time.sleep(2)   # to avoid being recognize as spam
    with requests.Session() as res:
        headers = {"User-Agent": 'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36'}
        response = res.get(url, timeout=5, headers = headers) #proxies={"http": current_proxy, "https": current_proxy})
    soup = BeautifulSoup(response.text, "html.parser")
    if response.status_code == requests.codes.ok:
        result = soup.find('title')
        print(result)
        return result
    else:
        if response.status_code == '503':
            print("sigh")
            return

def proc_result(result):
    if result == None:
        return 
    else:
        string = ''
        removed = result.text.strip()
        name = removed.split(":")
        string += name[0]
        if name[1] == ' Amazon.co.uk':
            return name[0]
        else:
            string += name[1]
            return string
        
prod = []
for i in range(len(url_list)):
    result = get_product(url_list[i])
    if result == 'Robot Check':
        print("Siao liao")
        
    else:
        prod_name = proc_result(result)
        print(prod_name)
        prod.append(prod_name)

print(prod)

http://www.amazon.co.uk/Dungeon-Masters-Dungeons-Dragons-Accessories
None
http://www.amazon.co.uk/Monster-Manual-Dungeons-Rulebook-Rulebooks/dp/0786965614
<title>Monster Manual: A Dungeons &amp; Dragons Core Rulebook: Amazon.co.uk: Wizards of the Coast: 8601410683740: Books</title>
Monster Manual A Dungeons & Dragons Core Rulebook
http://www.amazon.co.uk/Dungeon-Masters-Dungeons-Dragons-Rulebooks/dp/0786965622
<title>Dungeon Master's Guide (Dungeons &amp; Dragons Core Rulebooks): Amazon.co.uk: Wizards of the Coast: 8601416371511: Books</title>
Dungeon Master's Guide (Dungeons & Dragons Core Rulebooks)
http://www.amazon.co.uk/Dungeons-Dragons-Players-Handbook-Rulebooks/dp/0786965606
<title>Dungeons &amp; Dragons Player's Handbook (Dungeons &amp; Dragons Core Rulebooks): Amazon.co.uk: 8601421717694: Books</title>
Dungeons & Dragons Player's Handbook (Dungeons & Dragons Core Rulebooks)
[None, 'Monster Manual A Dungeons & Dragons Core Rulebook', "Dungeon Master's Guide (Dungeons & Dragons 

In [ ]:
print(alist[1265], alist[1238])